# Application for estimating the grades of portuguese students

## 1. Import all needed libraries

In [1]:
import tkinter as tk
import pandas as pd

from tkinter import ttk
from tkinter import filedialog as fd

from app.preprocessing import PreprocessingCSV

## 2. Settings for the application

## 3. Define Application

In [2]:
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        
        self._path: str = ""
        
        self.title("Student Grade Prediction")
        self.geometry("400x400")
        self.resizable(False, False)

        # Open CSV file
        self.open_csv_button = ttk.Button(self, text="Load Students CSV file", command=self.select_csv)
        self.open_csv_button.pack()
        
        # Predict button
        self.predict_button = ttk.Button(self, text="Predict", command=self.predict)
        self.predict_button.pack()
        
    def reset(self):
        self._path = ""
        print("Reset")
    
    def select_csv(self):
        accepted_formats = [("CSV files", "*.csv"), ("All files", "*.*")]
        self._path = fd.askopenfilename(title="Select CSV file", filetypes=accepted_formats)
        print(self._path)
        
    def predict(self):
        if self._path:
            preprocessing = PreprocessingCSV(self._path, sep=";")
            preprocessing.process()
            data = preprocessing.processed_data
            print(data.info(verbose=True))
        else:
            print("No CSV file selected")
    

## Start App

In [3]:
if __name__ == "__main__":
    app = App()
    app.mainloop()

/home/tom/projects/probability-ki/data/student-por_2.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 31 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   school         599 non-null    object 
 1   sex            599 non-null    object 
 2   age            599 non-null    object 
 3   address        599 non-null    object 
 4   famsize        599 non-null    object 
 5   Pstatus        599 non-null    object 
 6   reason         599 non-null    object 
 7   guardian       599 non-null    object 
 8   traveltime     599 non-null    object 
 9   studytime      599 non-null    object 
 10  failures       599 non-null    object 
 11  activities     599 non-null    object 
 12  nursery        599 non-null    object 
 13  higher         599 non-null    object 
 14  internet       599 non-null    object 
 15  romantic       599 non-null    object 
 16  famrel         599 non-null    object 
 1